In [1]:
import numpy as np
import xarray as xr
import pickle
import os



import sys
sys.path.append('/home/ascherrmann/scripts/')
import helper
sys.path.append('/home/freimax/msc_thesis/scripts/helpers/')
import data_preprocessing_helpers

In [2]:
####
## This is only for testing purposes
#-----------------------------------

# sp = '/net/helium/atmosdyn/freimax/data_msc/IFS-17/ctraj'
# f = open(f'{sp}/CaseStudy-CYC-Apr17.txt','rb')
# gd = pickle.load(f)
# f.close()


# for k in gd.keys():
#     print(k)#,mid)
#     mid = np.where(gd[k]['hSLP']==0)[0][0]
#     print(gd[k])
#     print(gd[k].keys())

In [5]:
for j in gd.keys():
    print(j)        

19


In [4]:
sp = '/net/helium/atmosdyn/freimax/data_msc/IFS-17/ctraj'
f = open(f'{sp}/CaseStudy-CYC-Apr17.txt','rb')
gd = pickle.load(f)
f.close()

Lat = np.round(np.linspace(0,90,226),2)
Lon = np.round(np.linspace(-180,180,901),2)


# for j in gd.keys():
j = 19      # The loop only consists of one iteration
mid = np.where(gd[j]['hSLP']==0)[0][0]
clat = gd[j]['clat'][mid].astype(int)
clon = gd[j]['clon'][mid].astype(int)
t = gd[j]['dates'][mid]

In [54]:
yyyy = int(t[0:4])
MM = int(t[4:6])
DD = int(t[6:8])
hh = int(t[9:])

print(t)
ds = xr.open_dataset(f'/net/thermo/atmosdyn/atroman/PAC1d/cdf/2017/04/S{t}', drop_variables=['P','TH','THE','RH','PVRCONVT','PVRCONVM','PVRTURBT','PVRTURBM','PVRLS','PVRCOND','PVRSW','PVRLWH','PVRLWC','PVRDEP','PVREVC','PVREVR','PVRCONV', 'PVRTURB', 'DTHDX', 'DTHDY' ,'PVRSUBI','PVRSUBS','PVRMELTI','PVRMELTS','PVRFRZ','PVRRIME','PVRBF'])
# s = xr.open_dataset('/net/thermo/atmosdyn2/atroman/PACaccu/cdf/P'+t, drop_variables=['ttot','tdyn','tsw','tlw','tmix','tconv','tls','tcond','tdep','tbf','tevc','tfrz','trime','tce'])
ds = ds.squeeze()

20170410_16


In [55]:
## Calculate 3d pressure field on model-levelsfor entire world
ds['pres'] = data_preprocessing_helpers.PRES_3d(ds, ds['PV'])

In [24]:
# Initialize arrays
pt = np.array([])
plat = np.array([])
plon = np.array([])

# Loop over every point in the circle and extract all valid pressure levels
for lon, lat in zip(clon, clat):
    
    # Extract and filter pressure level at lon/lat point
    P = ds.isel(dimx_PS=lon, dimy_PS=lat)
    valid_P = P.pres[(P.pres >= 700) & (P.pres <= 975)]# & (P.PV>=0.75)] 
    
    # Append values to the specific arrays
    pt = np.append(pt, valid_P)
    plat = np.append(np.round(plat,1), Lat[[lat] * len(valid_P)])
    plon = np.append(np.round(plon,1), Lon[[lon] * len(valid_P)])

# Stack arrays into a matrix
result_new = np.column_stack((np.zeros_like(pt), plon, plat, pt))

In [25]:
result_new

array([[  0.        , 162.4       ,  45.6       , 974.77008057],
       [  0.        , 162.4       ,  45.6       , 968.85412598],
       [  0.        , 162.4       ,  45.6       , 962.44421387],
       ...,
       [  0.        , 164.4       ,  48.8       , 749.25817871],
       [  0.        , 164.4       ,  48.8       , 730.5324707 ],
       [  0.        , 164.4       ,  48.8       , 711.24957275]])

In [16]:
result_new.shape

(2085, 4)

In [62]:
s = xr.open_dataset(f'/net/thermo/atmosdyn/atroman/PAC1d/cdf/2017/04/S{t}', drop_variables=['P','TH','THE','RH','PVRCONVT','PVRCONVM','PVRTURBT','PVRTURBM','PVRLS','PVRCOND','PVRSW','PVRLWH','PVRLWC','PVRDEP','PVREVC','PVREVR','PVRCONV', 'PVRTURB', 'DTHDX', 'DTHDY' ,'PVRSUBI','PVRSUBS','PVRMELTI','PVRMELTS','PVRFRZ','PVRRIME','PVRBF'])
PS = s.PS.values[0,0,clat,clon]
pv = s.PV.values

In [63]:
s.PS.values[0,0,:,:].shape

(451, 901)

In [77]:
clat.shape

(95,)

In [78]:
clat

array([114, 114, 114, 114, 114, 115, 115, 115, 115, 115, 115, 115, 115,
       115, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 117,
       117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 118,
       118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 119,
       119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 120,
       120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 121,
       121, 121, 121, 121, 121, 121, 121, 121, 121, 121, 122, 122, 122,
       122, 122, 122, 122])

In [79]:
clon

array([856, 857, 858, 859, 860, 854, 855, 856, 857, 858, 859, 860, 861,
       862, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 852,
       853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 852,
       853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 852,
       853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 852,
       853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 853,
       854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 855, 856, 857,
       858, 859, 860, 861])

In [64]:
s.PS.values[0,0,clat,clon].shape

(95,)

In [71]:
for l in range(len(clat)):
    P = helper.modellevel_to_pressure(PS[l])
    pid = np.where((P>=700) & (P<=975) & (pv[0,:,clat[l],clon[l]]>=0.75))[0]
    for i in pid:
            pt = np.append(pt,P[i])
            plat = np.append(plat,Lat[clat[l]])
            plon = np.append(plon,Lon[clon[l]])
            # print(clat[l])

save = np.zeros((len(pt),4))
save[:,1] = plon
save[:,2] = plat
save[:,3] = pt

In [74]:
save

array([[  0.        , 162.4       ,  45.6       , 974.77008057],
       [  0.        , 162.4       ,  45.6       , 968.85412598],
       [  0.        , 162.4       ,  45.6       , 962.44421387],
       ...,
       [  0.        , 164.4       ,  48.8       , 749.25817871],
       [  0.        , 164.4       ,  48.8       , 730.5324707 ],
       [  0.        , 164.4       ,  48.8       , 711.24957275]])

In [86]:
## Save the file
np.savetxt(f'{sp}/trastart-mature-{t}-ID-{j:06d}.txt' ,save,fmt='%f', delimiter=' ', newline='\n')
print(f'File saved in:\n\t{sp}/trastart-mature-{t}-ID-{j:06d}.txt' )

File saved in:
	/net/helium/atmosdyn/freimax/data_msc/IFS-17/ctraj/trastart-mature-20170410_16-ID-000019.txt
